<a href="https://colab.research.google.com/github/amitsiwach/Deep_Learning_notebooks/blob/main/Batch_Normalization_%26_MLflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import mlflow
import mlflow.tensorflow
import mlflow.keras
plt.style.use("fivethirtyeight")
%load_ext tensorboard

In [2]:
(X_train_full, y_train_full), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [3]:
tf.random.set_seed(42)
np.random.seed(42)

LAYERS = [ tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(300, kernel_initializer="he_normal"),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Dense(100, kernel_initializer="he_normal"),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Dense(10, activation="softmax")]


model = tf.keras.models.Sequential(LAYERS)

In [4]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [5]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid), verbose=2)

Epoch 1/10
1719/1719 - 7s - loss: 1.2819 - accuracy: 0.6229 - val_loss: 0.8886 - val_accuracy: 0.7160 - 7s/epoch - 4ms/step
Epoch 2/10
1719/1719 - 5s - loss: 0.7955 - accuracy: 0.7362 - val_loss: 0.7130 - val_accuracy: 0.7658 - 5s/epoch - 3ms/step
Epoch 3/10
1719/1719 - 4s - loss: 0.6816 - accuracy: 0.7720 - val_loss: 0.6427 - val_accuracy: 0.7898 - 4s/epoch - 2ms/step
Epoch 4/10
1719/1719 - 4s - loss: 0.6217 - accuracy: 0.7944 - val_loss: 0.5900 - val_accuracy: 0.8064 - 4s/epoch - 2ms/step
Epoch 5/10
1719/1719 - 4s - loss: 0.5832 - accuracy: 0.8074 - val_loss: 0.5582 - val_accuracy: 0.8202 - 4s/epoch - 2ms/step
Epoch 6/10
1719/1719 - 4s - loss: 0.5553 - accuracy: 0.8156 - val_loss: 0.5350 - val_accuracy: 0.8238 - 4s/epoch - 2ms/step
Epoch 7/10
1719/1719 - 4s - loss: 0.5338 - accuracy: 0.8225 - val_loss: 0.5157 - val_accuracy: 0.8304 - 4s/epoch - 2ms/step
Epoch 8/10
1719/1719 - 5s - loss: 0.5173 - accuracy: 0.8273 - val_loss: 0.5079 - val_accuracy: 0.8284 - 5s/epoch - 3ms/step
Epoch 9/

### Batch Normalization approach -1

In [6]:
del model

In [7]:
LAYERS_BN = [
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(300, activation="relu"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(10, activation="softmax")
]

model = tf.keras.models.Sequential(LAYERS_BN)

In [8]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 batch_normalization (BatchN  (None, 784)              3136      
 ormalization)                                                   
                                                                 
 dense_3 (Dense)             (None, 300)               235500    
                                                                 
 batch_normalization_1 (Batc  (None, 300)              1200      
 hNormalization)                                                 
                                                                 
 dense_4 (Dense)             (None, 100)               30100     
                                                                 
 batch_normalization_2 (Batc  (None, 100)             

In [9]:
784 * 4 # mean, variance, gamma and Beta (out of these 4, only Gamma & Beta are Trainable )

3136

In [10]:
300 * 4

1200

In [11]:
100 *4 

400

In [12]:
3136 + 1200 + 400

4736

In [13]:
4736 / 2 # (only Gamma & Beta are Trainable )

2368.0

In [14]:
266610 + 2368.0

268978.0

In [15]:
266610 + 4736

271346

In [16]:
model.layers

In [17]:
bn1 = model.layers[1]

In [18]:
for variable in bn1.variables:
  print(variable.name, variable.trainable)

batch_normalization/gamma:0 True
batch_normalization/beta:0 True
batch_normalization/moving_mean:0 False
batch_normalization/moving_variance:0 False


In [19]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [20]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid), verbose=2)

Epoch 1/10
1719/1719 - 7s - loss: 0.8294 - accuracy: 0.7221 - val_loss: 0.5539 - val_accuracy: 0.8164 - 7s/epoch - 4ms/step
Epoch 2/10
1719/1719 - 7s - loss: 0.5704 - accuracy: 0.8035 - val_loss: 0.4791 - val_accuracy: 0.8380 - 7s/epoch - 4ms/step
Epoch 3/10
1719/1719 - 6s - loss: 0.5162 - accuracy: 0.8213 - val_loss: 0.4424 - val_accuracy: 0.8492 - 6s/epoch - 3ms/step
Epoch 4/10
1719/1719 - 6s - loss: 0.4789 - accuracy: 0.8315 - val_loss: 0.4213 - val_accuracy: 0.8558 - 6s/epoch - 3ms/step
Epoch 5/10
1719/1719 - 6s - loss: 0.4548 - accuracy: 0.8407 - val_loss: 0.4050 - val_accuracy: 0.8614 - 6s/epoch - 4ms/step
Epoch 6/10
1719/1719 - 6s - loss: 0.4387 - accuracy: 0.8445 - val_loss: 0.3932 - val_accuracy: 0.8632 - 6s/epoch - 3ms/step
Epoch 7/10
1719/1719 - 6s - loss: 0.4255 - accuracy: 0.8503 - val_loss: 0.3830 - val_accuracy: 0.8646 - 6s/epoch - 3ms/step
Epoch 8/10
1719/1719 - 6s - loss: 0.4124 - accuracy: 0.8536 - val_loss: 0.3759 - val_accuracy: 0.8668 - 6s/epoch - 4ms/step
Epoch 9/

### Batch Normalization approach 2

In [21]:
del model

In [22]:
LAYERS_BN_BIAS_FALSE = [
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(300, use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.Dense(100, use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.Dense(10, activation="softmax")
]

model = tf.keras.models.Sequential(LAYERS_BN_BIAS_FALSE)

In [23]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [24]:
mlflow.tensorflow.autolog()

with mlflow.start_run(run_name="MLflow on Colab"):
  history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 9s 5ms/step - loss: 1.0346 - accuracy: 0.6739 - val_loss: 0.6680 - val_accuracy: 0.7886
Epoch 2/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.6757 - accuracy: 0.7819 - val_loss: 0.5537 - val_accuracy: 0.8212
Epoch 3/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.5961 - accuracy: 0.8021 - val_loss: 0.4996 - val_accuracy: 0.8352
Epoch 4/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.5456 - accuracy: 0.8176 - val_loss: 0.4655 - val_accuracy: 0.8458
Epoch 5/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.5140 - accuracy: 0.8251 - val_loss: 0.4419 - val_accuracy: 0.8508
Epoch 6/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.4915 - accuracy: 0.8308 - val_loss: 0.4238 - val_accuracy: 0.8540
Epoch 7/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.4741 - accuracy: 0.8377 - val_loss: 0.4103 - val_accuracy:

2022/11/17 05:44:06 WARNING mlflow.utils.requirements_utils: Found jaxlib version (0.3.22+cuda11.cudnn805) contains a local version label (+cuda11.cudnn805). MLflow logged a pip requirement for this package as 'jaxlib==0.3.22' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.


In [25]:
get_ipython().system_raw("mlflow ui --port 5000 &")

In [26]:
# !pip install pyngrok
from pyngrok import ngrok

# Terminate open tunnels if exist
ngrok.kill()

In [27]:
# Setting the authtoken (optional)
# Get your authtoken from https://dashboard.ngrok.com/auth
NGROK_AUTH_TOKEN = "2Hf3cPKgE4umfpIOOxwXr2SsGTr_7dk2Q3VMkwFf922HoqyZx"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

In [28]:
# Open an HTTPs tunnel on port 5000 for http://localhost:5000
ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)
print("MLflow Tracking UI:", ngrok_tunnel.public_url)

MLflow Tracking UI: https://88aa-35-185-244-55.ngrok.io
